# 实例1 表的联合查询

In [ ]:
# 导入所需库
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime

In [ ]:
# 导入数据文件（excel）
# 6month.xlsx是6月份用户订单数据，7month.xlsx是7月份用户订单数据，注册用户数据.xls是用户信息数据
df_6 = pd.read_excel("6month.xlsx")
df_7 = pd.read_excel("7month.xlsx")
df_users = pd.read_excel(u"注册用户数据.xls")

In [ ]:
# 定义时间类型变量
six_m = pd.to_datetime('2017-06-01')
senven_m = pd.to_datetime('2017-07-01')
eight_m = pd.to_datetime('2017-08-01')

In [ ]:
# 在用户数据表中筛选6月（7月）注册用户
user_6 = df_users[(df_users[u'注册时间'] >= six_m) & (df_users[u'注册时间'] < senven_m)][[u'注册时间',u'药店名称']]
user_7 = df_users[(df_users[u'注册时间'] >= senven_m) & (df_users[u'注册时间'] < eight_m)][[u'注册时间',u'药店名称']]

In [ ]:
# 连接筛选后的表和6月（7月）订单表，找出6月（7月）注册用户的订单
sheet_6 = pd.merge(df_6, user_6, left_on=u'下单用户', right_on=u'药店名称')
sheet_7 = pd.merge(df_7, user_7, left_on=u'下单用户', right_on=u'药店名称')

# 实例2 分组排序并获取每个分组前三条数据

In [ ]:
# 接实例1
# 修改时间数据类型，便于进行时间比较
sheet_6[u'下单时间'] = pd.to_datetime(sheet_6[u'下单时间'])
sheet_7[u'下单时间'] = pd.to_datetime(sheet_7[u'下单时间'])

In [ ]:
def top(df, n=3, column=u'下单时间'):
    return df.sort_values(by=column)[0:n]

In [ ]:
s6 = sheet_6.groupby(u'下单用户').apply(top)

In [ ]:
s7 = sheet_7.groupby(u'下单用户').apply(top)

In [ ]:
# 导出表格到excel文件
writer = pd.ExcelWriter('output.xlsx')
sheet_6.to_excel(writer,u'6月新用户订单')
sheet_7.to_excel(writer,u'7月新用户订单')
s6.to_excel(writer,u'6月新用户前三笔订单')
s7.to_excel(writer,u'7月新用户前三笔订单')
writer.save()

In [ ]:
# 分别获取每个用户前三笔数据

In [ ]:
def get_one(df, n=0, column=u'下单时间'):
    return df.sort_values(by=column)[n:n+1]

In [ ]:
s6_1 = sheet_6.groupby(u'下单用户').apply(get_one, n=0)
s6_2 = sheet_6.groupby(u'下单用户').apply(get_one, n=1)
s6_3 = sheet_6.groupby(u'下单用户').apply(get_one, n=2)

In [ ]:
s7_1 = sheet_7.groupby(u'下单用户').apply(get_one, n=0)
s7_2 = sheet_7.groupby(u'下单用户').apply(get_one, n=1)
s7_3 = sheet_7.groupby(u'下单用户').apply(get_one, n=2)

In [ ]:
writer = pd.ExcelWriter('output.xlsx')
s6_1.to_excel(writer,u'6月新用户第一笔订单')
s6_2.to_excel(writer,u'6月新用户第二笔订单')
s6_3.to_excel(writer,u'6月新用户第三笔订单')
s7_1.to_excel(writer,u'7月新用户第一笔订单')
s7_2.to_excel(writer,u'7月新用户第二笔订单')
s7_3.to_excel(writer,u'7月新用户第三笔订单')
writer.save()

# 实例3 合并拆单数据，并将数据更新到母单

In [ ]:
# 导入数据
df = pd.read_excel("March.xlsx")

In [ ]:
# 去重
df = df.drop_duplicates(u'订单编号')

In [ ]:
# 查看列选项3种方式

# 利用去重函数
df[u'状态'].drop_duplicates()

# 用value_counts也可以实现查看，函数表示各值出现的频率
df[u'状态'].value_counts()

# 用unique查看列选项，不过如果选项是中文会显示编码，不易识别。因而下面用循环打印显示中文
for x in df[u'状态'].unique():
    print x

In [ ]:
# 筛选表
status = [u'已完成', u'已拆单', u'订单审核中']
df1 = df[df[u'状态'].isin(status)]

In [ ]:
# 分别提取母表和子表
df_sub = df1[df1[u'订单编号'].str.len()>20]
df_mo = df1[df1[u'订单编号'].str.len() == 20]

In [ ]:
# 处理字表字段
df_sub[u'订单编号'] = df_sub[u'订单编号'].map(lambda x:x[0:-2])

In [ ]:
# 字表分组求和
df_sub_bak = df_sub.copy() # 备份子单数据，应对新需求
df_sub = df_sub.groupby(u'订单编号')[u'总金额',u'优惠金额'].sum()

In [ ]:
# 根据子单索引（订单编号）修改母单中对应列的值（总金额和优惠金额）
def func_rep1(r):
    # 利用get_loc函数，找到子单订单编号在母单中的索引（get_loc是Index类型的函数，根据值查找对应的索引）
    # 注意异常处理，当查询失败时返回异常（有异常直接跳过）
    try:
        i = pd.Index(df_mo[u'订单编号']).get_loc(r)
    except:
        pass
    else:
        df_mo.loc[i,[u'总金额']] = df_sub.loc[r,[u'总金额']]
        df_mo.loc[i,[u'优惠金额']] = df_sub.loc[r,[u'优惠金额']]
df_sub.index.map(func_rep1)

In [ ]:
# 另外一种思路
# 将母单的索引设置为订单编号，以保持跟子单的类型一致。然后根据索引修改对应值
# inplace参数表示直接修改表，而不是返回一张新表
df_mo.set_index(u'订单编号', inplace=True)

# 函数首先判断订单编号是否在母单索引中。如果在其中则替换相应值
def func_rep1(r):
    if r in df_mo.index:
        df_mo.loc[r,[u'总金额']] = df_sub.loc[r, [u'总金额']]
        df_mo.loc[r,[u'优惠金额']] = df_sub.loc[r, [u'优惠金额']]
df_sub.index.map(func_rep1)

In [ ]:
# 如果要求将子单中存在而母单不存在的数据加入母单，使用下面的函数（对应上面第一种思路）
def func_rep2(r):
    # 利用get_loc函数，找到子单订单编号在母单中的索引（get_loc是Index类型的函数，根据值查找对应的索引）
    # 注意异常处理，当查询失败时返回异常（有异常直接跳过）
    i = -1
    try:
        i = pd.Index(df_mo[u'订单编号']).get_loc(r)
    except:
        # 找到子单数据合并到母单
        tmp = df_sub_bak[df_sub_bak[u'订单编号'] == r]
        df_mo.loc[df_mo.index.max()+1] = tmp.iloc[0]
    else:
        df_mo.loc[i,[u'总金额']] = df_sub.loc[r,[u'总金额']]
        df_mo.loc[i,[u'优惠金额']] = df_sub.loc[r,[u'优惠金额']]
df_sub.index.map(func_rep2)

In [ ]:
# 函数首先判断订单编号是否在母单索引中。如果在其中则替换相应值，否则将整条数据插入母单（对应上面思路2）
df_mo.set_index(u'订单编号', inplace=True)
def func_rep2(r):
    if r in df_mo.index:
        df_mo.loc[r,[u'总金额']] = df_sub.loc[r, [u'总金额']]
        df_mo.loc[r,[u'优惠金额']] = df_sub.loc[r, [u'优惠金额']]
    else:
        df_mo.loc[r] = df_sub_bak[df_sub_bak[u'订单编号'] == r].iloc[0]
df_sub.index.map(func_rep2)